# IBM Data Science Capstone Project (Coursera)

## Final Project - The Battle of Neighborhoods

### To Open a new shoping mall in Bangalore(India), the following steps are required for the project.

 1. Build a dataframe of neighborhoods in Bangalore. 
 2. Get the latitude and longitude values of the neighborhoods.
 3. Obtain the venue data for the neighborhoods from Foursquare API.
 4. Explore and Analyze the neighborhoods in Bangalore.
 5. Cluster Neighborhoods and select the best cluster to open a new shopping mall.

In [1]:
!conda install -c conda-forge geopy --yes
get_ipython().system(u' pip install beautifulsoup4')
!pip install geocoder

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         395 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

### 1. Importing the Libraries needed to build the dataframe.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


#### Scraping of the data from Wikipedia page into a DataFram

In [3]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

# create the list to store table data
neighborhood_list = []

#### Using of BeautifulSoup to transform the data in the table on the Wikipedia page into dataframe

In [4]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhood_list.append(row.text)

In [5]:
bl_df = pd.DataFrame({"Neighborhood": neighborhood_list})

bl_df.head()

,Neighborhood
0,List of areas in Bangalore Cantonment
1,List of areas in Bengaluru Pete
2,List of neighbourhoods in Bangalore
3,Adugodi
4,"Agara, Bangalore"


In [6]:
# drop cells with a Neighborhood that is Not a Neighborhood
bl_df = bl_df[bl_df.Neighborhood != "List of areas in Bangalore Cantonment"].reset_index(drop=True)
bl_df = bl_df[bl_df.Neighborhood !="List of areas in Bengaluru Pete"].reset_index(drop=True)
bl_df = bl_df[bl_df.Neighborhood !="List of neighbourhoods in Bangalore"].reset_index(drop=True)
bl_df.head()

,Neighborhood
0,Adugodi
1,"Agara, Bangalore"
2,Ananthnagar
3,Anjanapura
4,Arekere


In [7]:
# print the number of rows of the  dataframe
bl_df.shape

(128, 1)

### 2. Get the latitude and longitude values of the neighborhoods.

In [8]:
# define a function to get coordinates

def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in bl_df["Neighborhood"].tolist() ]

In [63]:
coords

[[12.944020000000023, 77.60800000000006],
 [12.842830000000049, 77.48759000000007],
 [12.954080000000033, 77.54135000000008],
 [12.858110000000067, 77.55909000000008],
 [12.885680000000036, 77.59668000000005],
 [12.963480000000061, 77.61297000000008],
 [13.02753000000007, 77.65049000000005],
 [13.07728748449913, 77.65760577036913],
 [13.044710000000066, 77.55008000000004],
 [12.922310000000039, 77.56988000000007],
 [13.019643510687336, 77.65469211693214],
 [12.938980000000072, 77.57137000000006],
 [12.992220000000032, 77.53444000000007],
 [12.927350000000047, 77.67185000000006],
 [13.083010000000058, 77.54779000000008],
 [12.900090000000034, 77.60433000000006],
 [12.903080000000045, 77.62444000000005],
 [12.817530000000033, 77.67879000000005],
 [12.993330000000071, 77.66123000000005],
 [12.91488000000004, 77.61004000000008],
 [12.951940000000036, 77.54445000000004],
 [12.913110000000074, 77.71565000000004],
 [12.959700296061198, 77.57131179631511],
 [12.794010000000071, 77.700150000000

In [10]:
# create temporary dataframe to populate the coordinates of the neighbourhood into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [11]:
# merge the coordinates into the bl dataframe
bl_df['Latitude'] = df_coords['Latitude']
bl_df['Longitude'] = df_coords['Longitude']

In [12]:
# print the number of rows of the  dataframe
bl_df.shape

(128, 3)

In [13]:
bl_df.head()

,Neighborhood,Latitude,Longitude
0,Adugodi,12.94402,77.60800
1,"Agara, Bangalore",12.84283,77.48759
2,Ananthnagar,12.95408,77.54135
3,Anjanapura,12.85811,77.55909
4,Arekere,12.88568,77.59668


In [15]:
# save the DataFrame as CSV file
bl_df.to_csv("bl_df.csv", index=False)

### 3. Obtain the venue data for the neighborhoods from Foursquare API.

In [14]:
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


#### Create a map of Bangalore with neighborhoods superimposed on top.¶

In [23]:
# create map of Bangalore using latitude and longitude values
map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(bl_df['Latitude'], bl_df['Longitude'], bl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore)  
    
map_bangalore

In [18]:
# save the map as HTML file
map_bangalore.save('map_bangalore.html')

### 4. Explore and Analyze the neighborhoods in Bangalore.

#### Explore the neighborhoods using Foursquare API

In [15]:
CLIENT_ID = 'YYSO2QK3GLZQ3B1WZOFZEVL3G4QJP5JJJZ0OQ3UN5XHZVAQX' # Foursquare ID
CLIENT_SECRET = '3CO445BQDK0OGAAMLDY0EA5BQCZSCJIQAWZ3CA2LDHMAEKGV' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YYSO2QK3GLZQ3B1WZOFZEVL3G4QJP5JJJZ0OQ3UN5XHZVAQX
CLIENT_SECRET:3CO445BQDK0OGAAMLDY0EA5BQCZSCJIQAWZ3CA2LDHMAEKGV


In [16]:
# Let's explore the first neighborhood in our dataframe.
bl_df.loc[0, 'Neighborhood']

'Adugodi'

In [17]:
# Get the neighborhood's latitude and longitude values.
neighborhood_latitude = bl_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = bl_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = bl_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Adugodi are 12.944020000000023, 77.60800000000006.


##### Now, let's get the top 100 venues that are within a radius of 1000 meters.

In [18]:
# explore the top 100 venues
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
venues = []

for lat, long, neighborhood in zip(bl_df['Latitude'], bl_df['Longitude'], bl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [19]:
# clean the json and structure it into a pandas dataframe
bl_venues = pd.DataFrame(venues)

# define the column names
bl_venues.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(bl_venues.shape)


(2939, 7)


In [20]:
bl_venues.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adugodi,12.94402,77.608,The Chocolate Room,12.935505,77.608689,Café
1,Adugodi,12.94402,77.608,Cafe Coffee Day,12.936361,77.610583,Café
2,Adugodi,12.94402,77.608,Cafe Aloha,12.935651,77.608078,Café
3,Adugodi,12.94402,77.608,Cafe Coffee Day,12.943541,77.600029,Café
4,Adugodi,12.94402,77.608,Malabar Bay,12.935273,77.607618,Kerala Restaurant


In [21]:
bl_venues.to_csv("bl_venues.csv", index=False)

In [26]:
# how many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(bl_venues.shape[0]))

2962 venues were returned by Foursquare.


#### Calculate how many venues were returned for each neighborhood?

In [27]:
bl_venues.groupby('Neighborhood').count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,8,8,8,8,8,8
Ananthnagar,8,8,8,8,8,8
Anjanapura,3,3,3,3,3,3
Arekere,39,39,39,39,39,39
Austin Town,58,58,58,58,58,58
BTM Layout,48,48,48,48,48,48
Babusapalya,11,11,11,11,11,11
"Bagalur, Bangalore Urban",1,1,1,1,1,1
Bahubalinagar,4,4,4,4,4,4


#### How many unique categories can be curated from all the returned venues?

In [28]:
print('There are {} uniques categories.'.format(len(bl_venues['VenueCategory'].unique())))

There are 221 uniques categories.


In [29]:
# print out the list of categories
bl_venues['VenueCategory'].unique()[:50]

array(['Café', 'Kerala Restaurant', 'Bus Station', 'Spa',
       'Mughlai Restaurant', 'Indian Restaurant', 'Travel & Transport',
       'Metro Station', 'Smoke Shop', 'Gym', 'Nightclub', 'Pool', 'ATM',
       'Campground', 'Sporting Goods Shop', 'Ice Cream Shop', 'BBQ Joint',
       'Chinese Restaurant', 'Pizza Place', 'Department Store',
       'Liquor Store', 'Fast Food Restaurant', 'Burger Joint', 'Bar',
       'Supermarket', 'Gym / Fitness Center', 'Bus Line',
       'Dumpling Restaurant', 'Breakfast Spot', 'Neighborhood',
       'Badminton Court', 'Food Truck', 'Diner',
       'South Indian Restaurant', 'Sandwich Place',
       'Mediterranean Restaurant', 'Lake', 'Cocktail Bar',
       'Soccer Stadium', 'Brewery', 'Pub', 'Shopping Mall',
       'Toy / Game Store', 'Hotel', 'Italian Restaurant', 'Tea Room',
       'Korean Restaurant', 'Steakhouse', 'Donut Shop',
       'Middle Eastern Restaurant'], dtype=object)

In [22]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in bl_venues['VenueCategory'].unique()

True

#### Analyze Each Neighborhood

In [26]:
# one hot encoding
bl_onehot = pd.get_dummies(bl_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bl_onehot['Neighborhood'] = bl_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bl_onehot.columns[-1]] + list(bl_onehot.columns[:-1])
bl_onehot = bl_onehot[fixed_columns]

bl_onehot.head()

,Women's Store,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Candy Store,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indie Theater,Indoor Play Area,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tailor Shop,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Travel & Transport,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [27]:
# Examine the new dataframe size.
bl_onehot.shape

(2939, 221)

#### let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [28]:
bl_grouped = bl_onehot.groupby('Neighborhood').mean().reset_index()
print(bl_grouped.shape)
bl_grouped

(126, 221)


,Neighborhood,Women's Store,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Candy Store,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indie Theater,Indoor Play Area,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tailor Shop,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Travel & Transport,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Adugodi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.571429,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000

#### Now, Let's create a new dataframe for Shopping Mall data.

In [29]:
bl_mall = bl_grouped[["Neighborhood","Shopping Mall"]]
bl_mall.head()

,Neighborhood,Shopping Mall
0,Adugodi,0.000000
1,Ananthnagar,0.000000
2,Anjanapura,0.000000
3,Arekere,0.000000
4,Austin Town,0.016949


### 5. Cluster Neighborhoods and select the best cluster to open a new shopping mall.

#### Cluster Neighborhoods (Run k-means to cluster the neighborhood into 5 clusters).

In [30]:
# set number of clusters
kclusters = 5

bl_clustering = bl_mall.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 3, 0, 0, 0, 4, 3], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
# add clustering labels

bl_merged = bl_mall

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bl_merged["Cluster Labels"] = kmeans.labels_

bl_merged.head() # check the last columns!

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Shopping Mall,Cluster Labels
0,Adugodi,0.000000,0
1,Ananthnagar,0.000000,0
2,Anjanapura,0.000000,0
3,Arekere,0.000000,0
4,Austin Town,0.016949,3


In [34]:
# merge the bl_groupled with bl_mall to add latitude/longitude for each neighborhood
bl_merged = bl_merged.join(bl_df.set_index("Neighborhood"), on="Neighborhood")

print(bl_merged.shape)
bl_merged.head() # check the last columns!

(126, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adugodi,0.000000,0,12.94402,77.60800
1,Ananthnagar,0.000000,0,12.95408,77.54135
2,Anjanapura,0.000000,0,12.85811,77.55909
3,Arekere,0.000000,0,12.88568,77.59668
4,Austin Town,0.016949,3,12.96348,77.61297


In [35]:
# sort the results by Cluster Labels
print(bl_merged.shape)
bl_merged.sort_values(["Cluster Labels"])
bl_merged

(126, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adugodi,0.000000,0,12.944020,77.608000
1,Ananthnagar,0.000000,0,12.954080,77.541350
2,Anjanapura,0.000000,0,12.858110,77.559090
3,Arekere,0.000000,0,12.885680,77.596680
4,Austin Town,0.016949,3,12.963480,77.612970
5,BTM Layout,0.000000,0,12.914880,77.610040
6,Babusapalya,0.000000,0,13.027530,77.650490
7,"Bagalur, Bangalore Urban",0.000000,0,13.077287,77.657606
8,Bahubalinagar,0.166667,4,13.044710,77.550080
9,Banashankari,0.025000,3,12.922310,77.569880


#### Finally, let's visualize the resulting clusters.

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bl_merged['Latitude'], bl_merged['Longitude'], bl_merged['Neighborhood'], bl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

#### Finally Let's examine Clusters to select the best cluster to open a new shopping mall.

#### Cluster 1

In [139]:
bl_merged.loc[bl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adugodi,0.0,0,12.944020,77.608000
89,R. T. Nagar,0.0,0,13.024450,77.595900
87,Peenya,0.0,0,13.031850,77.526790
84,Nayandahalli,0.0,0,12.942050,77.521000
83,Nandini Layout,0.0,0,13.014810,77.538910
82,Nagarbhavi,0.0,0,12.956240,77.509360
80,"Murphy Town, Bangalore",0.0,0,12.980030,77.627760
79,Milk Colony,0.0,0,13.023254,77.579079
78,Mathikere,0.0,0,13.032350,77.558660
77,Mariyannapalya,0.0,0,13.049640,77.611820


#### Cluster 2

In [140]:
bl_merged.loc[bl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
15,Bilekahalli,0.026316,1,12.90009,77.60433
13,Bellandur,0.023810,1,12.92735,77.67185
18,Brookefield,0.033333,1,12.99333,77.66123
73,"Mahadevapura, Bangalore",0.027778,1,12.99409,77.66633
74,Mahalakshmi Layout,0.045455,1,13.01635,77.54481
25,Cooke Town,0.031250,1,13.00363,77.62283
120,"Whitefield, Bangalore",0.039216,1,12.97938,77.73372
76,Marathahalli,0.029412,1,12.95466,77.70752
88,Puttenahalli,0.032258,1,12.89434,77.58507
9,Banashankari,0.023256,1,12.92231,77.56988


#### Cluster 3

In [141]:
bl_merged.loc[bl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
102,"Shivajinagar, Bangalore",0.017857,2,12.987200,77.604010
100,Seshadripuram,0.011765,2,12.993550,77.579880
4,Austin Town,0.017241,2,12.963480,77.612970
50,J. P. Nagar,0.011494,2,12.908310,77.590240
86,Palace Guttahalli,0.013889,2,12.995505,77.574771
52,"Jayanagar, Bangalore",0.010000,2,12.928720,77.582810
64,"Kodihalli, Bangalore",0.010000,2,12.966200,77.649820
90,Rajajinagar,0.011905,2,13.005440,77.556930


#### Cluster 4

In [142]:
bl_merged.loc[bl_merged['Cluster Labels'] == 3]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
26,"Cox Town, Bangalore",0.095238,3,12.99594,77.62644
106,Suddaguntepalya,0.083333,3,12.98389,77.65476
71,Lingarajapuram,0.100000,3,13.00548,77.62597
85,Padmanabhanagar,0.080000,3,12.91814,77.55940


#### Cluster 5

In [143]:
bl_merged.loc[bl_merged['Cluster Labels'] == 4]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
81,Muthyalanagar,0.125,4,13.04103,77.55148
27,Dasarahalli,0.125,4,13.04224,77.51507


### Final Analysis and Observation:

Most of the shopping malls are in the central area of Bangalore, with the highest numbers in cluster 5, then in cluster 4 and moderate numbers in cluster 2 and 3. However, cluster 1 has very low number to totally no shopping mall in the neighborhoods.Hence it is advisable for the property developers to capitalize on these findings and open new shopping malls in Cluser 1 area.  As there are not many shopping malls in Cluster 1, the new shopping mall will be very successful as  there will be very little to no market competition. However, in Cluster 5 and cluster 4 it seems like there are already many shopping malls exist. Hence it is advisable to avoid those neighbourhoods.